CONNECT TO THE PROJECT ROOT

In [ ]:
import os
import sys

# Get the absolute path of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Navigate up one directory to get the project root
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the project root to sys.path if not already present
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to sys.path: {project_root}")


CREATE THE TIME SERIES DIRECTOR